In [1]:
import polars as pl
from pathlib import Path

# stats generated from ~1300 replays from 1/1/2023 - 4/30/2023
l_cancel = pl.read_parquet(Path(R".\Output\l_cancels.parquet"))
l_cancel.schema

{'date_time': Datetime(time_unit='us', time_zone='America/Chicago'),
 'slippi_version': Utf8,
 'match_id': Utf8,
 'match_type': Utf8,
 'game_number': Int64,
 'stage': Utf8,
 'duration': Duration(time_unit='ms'),
 'result': Utf8,
 'port': Utf8,
 'connect_code': Utf8,
 'character': Utf8,
 'costume': Utf8,
 'opnt_character': Utf8,
 'frame_index': Int64,
 'stocks_remaining': Int64,
 'l_cancel': Boolean,
 'trigger_input_frame': Int64,
 'during_hitlag': Boolean,
 'move': Utf8,
 'position': Utf8,
 'fastfall': Boolean}

In [2]:
# Total l cancels
l_cancel.filter(pl.col("character") == "FALCO").groupby("l_cancel").agg(pl.count())

l_cancel,count
bool,u32
false,3073
true,19673


In [3]:
# L cancel percent by character
l_cancel.groupby("character").agg(pl.col("l_cancel").mean())

character,l_cancel
str,f64
"""CAPTAIN_FALCON…",0.809929
"""FALCO""",0.864899
"""FOX""",0.807692
"""MARTH""",0.819138


In [4]:
# L cancel percentage by move
l_cancel.filter(pl.col("character") == "FALCO").groupby(pl.col("move")).agg([pl.col("l_cancel").mean()])

move,l_cancel
str,f64
"""BAIR""",0.787123
"""DAIR""",0.873635
"""NAIR""",0.903505
"""FAIR""",0.817797
"""UAIR""",0.865533


In [5]:
( # l cancel percentage based on landing location
    l_cancel.filter(pl.col("character") == "FALCO")
    .groupby(pl.col("position"))
    .agg([pl.col("l_cancel").mean(), pl.count()])
)

position,l_cancel,count
str,f64,u32
"""RIGHT_PLATFORM…",0.703523,1022
"""RANDALL""",0.375,8
"""LEFT_PLATFORM""",0.676749,1058
"""MAIN_STAGE""",0.892154,19769
"""TOP_PLATFORM""",0.672666,889


In [6]:
( # L cancel percent by opponent character, sorted by number of l cancel occurances
    l_cancel.filter(pl.col("character") == "FALCO")
    .groupby(pl.col("opnt_character"))
    .agg([pl.col("l_cancel").mean(), pl.count()])
    .sort(pl.col("count"), descending=True)
)

opnt_character,l_cancel,count
str,f64,u32
"""FOX""",0.866252,7073
"""FALCO""",0.867229,3909
"""CAPTAIN_FALCON…",0.880541,3181
"""MARTH""",0.864424,2884
"""SHEIK""",0.837912,1820
"""PEACH""",0.855053,752
"""JIGGLYPUFF""",0.841463,410
"""DR_MARIO""",0.872,375
"""LUIGI""",0.87062,371


In [7]:
( # L cancel percentage by stocks remaining
    l_cancel.filter(pl.col("character") == "FALCO")
    .groupby(pl.col("stocks_remaining"))
    .agg([pl.col("l_cancel").mean(), pl.count()])
)

stocks_remaining,l_cancel,count
i64,f64,u32
1,0.875221,3967
2,0.861803,5579
3,0.866894,6461
4,0.859475,6739
